# Libs

In [81]:
import pandas as pd
import numpy as np
import requests
import urllib3
import json
from bs4 import BeautifulSoup as bs
import ipywidgets as widgets
from ipywidgets import fixed

In [100]:
%config IPCompleter.greedy=True
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Lista dos Fundos Imobiliarios

In [83]:
# o header abaixo serve para mascarar a requisição semelhante a um web browser
header = {"User-Agent": "Chrome/84.0.4147.89","X-Requested-With": "XMLHttpRequest"}

# URL com todos os FIIs listados
url = 'https://fundamentus.com.br/fii_resultado.php'

# Requests via HTTP
r = requests.get(url, headers=header)

# Leitura do HTML entregue
fii = pd.read_html(r.text,decimal=',',thousands='.')

In [84]:
# o índice [0] contem toda a tabela dos fundos
fundos = fii[0]

# Dividend Yield vem como string. É necessário remover '%' e trocar ',' por '.'
fundos['Dividend yield 2']= fundos['Dividend Yield'].str.replace('%','').str.replace(',','.').astype(float)
fundos['Dividendo/cota'] = fundos['Cotação']*fundos['Dividend yield 2']/100

fundos

,Papel,Segmento,Cotação,FFO Yield,Dividend Yield,P/VP,Valor de Mercado,Liquidez,Qtd de imóveis,Preço do m2,Aluguel por m2,Cap Rate,Vacância Média,Dividend yield 2,Dividendo/cota
0,ABCP11,Shoppings,69.32,"3,50%","3,44%",0.78,845970000,198748,1,11904.40,603.35,"5,07%","12,00%",3.44,2.384608
1,AEFI11,Outros,174.90,"6,20%","0,00%",1.16,353582000,0,5,4351.27,285.52,"6,56%","0,00%",0.00,0.000000
2,AFCR11,Híbrido,111.32,"6,01%","5,38%",1.07,132167000,409510,0,0.00,0.00,"0,00%","0,00%",5.38,5.989016
3,AFHI11,Títulos e Val. Mob.,92.00,"0,03%","0,00%",0.96,164466000,362124,0,0.00,0.00,"0,00%","0,00%",0.00,0.000000
4,AFOF11,Títulos e Val. Mob.,224.00,"1,79%","0,00%",2.08,82235800,111274,0,0.00,0.00,"0,00%","0,00%",0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,XPML11,Shoppings,104.10,"1,76%","2,56%",0.95,1895420000,3990670,10,5370.14,169.91,"3,16%","5,43%",2.56,2.664960
272,XPPR11,Outros,63.80,"10,92%","9,32%",0.73,466772000,1942160,5,7402.94,785.12,"10,61%","46,51%",9.32,5.946160
273,XPSF11,Títulos e Val. Mob.,97.35,"1,66%","6,45%",0.99,421546000,1628830,0,0.00,0.00,"0,00%","0,00%",6.45,6.279075
274,XTED11,Lajes Corporativas,6.97,"-8,96%","0,00%",0.49,12494100,10529,1,1662.03,0.00,"0,00%","0,00%",0.00,0.000000


# Obtendo o IPCA

In [85]:
# OBTER IPCA acumulado 12 MESES

r = requests.get('https://www.melhorcambio.com/ipca',headers = header,verify=False)

# web scrapying para "quebrar o conteúdo do html"
soup = bs(r.content,'html.parser')

ipca = soup.find_all('input',{'id':'inp_calc_3'})

ipca = float(ipca[0]['value'].replace(',','.'))
ipca

6.76

# Obtendo o SPREAD do IPCA

- o "*spread*" é a diferença que o título mais longo do Tesouro está pagando de diferencial em relação ao IPCA
- Exemplo: IPCA+2050 = IPCA + 5% -> os 5% é o spread do titulo

In [86]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [87]:
# função que faz o crawling dos dados do site do Banco Central

def busca_titulos_tesouro_direto():
    url = 'https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv'
    df  = pd.read_csv(url, sep=';', decimal=',')
    df['Data Vencimento'] = pd.to_datetime(df['Data Vencimento'], dayfirst=True)
    df['Data Base']       = pd.to_datetime(df['Data Base'], dayfirst=True)
    return df

In [88]:
# Transformar o título em dataframe

titulos = busca_titulos_tesouro_direto()
titulos.sort_index(inplace=True)

In [89]:
# Obter o IPCA+ longo

vencimento = titulos.loc[(titulos['Tipo Titulo'] == 'Tesouro IPCA+'),'Data Vencimento'].max()
ipca_longo = titulos.loc[(titulos['Tipo Titulo'] == 'Tesouro IPCA+') & (titulos['Data Vencimento'] == vencimento)]

In [91]:
# Obter o spread que o título mais longo está pagando 

ipca_longo.sort_values('Data Base',ascending = False, inplace = True)
spread_ntnb = ipca_longo.iloc[0,3]
spread_ntnb

4.21

# Modelo de Gordon

Modelo de Gordon = $( Dividendo ) / Wacc $

$ Wacc = Spread.NTNB + IPCA + Spread.Mercado$

- **Tijolo** -> `IPCA = 0` pois a inflação já é reposta pelo patrimônio e reajuste dos contratos
- **CRI** -> `IPCA <> 0` (retorno é nominal)

- `Spread NTN-B`: spread do título IPCA mais longo
- `IPCA`: indice acumulado dos ultimos 12 meses
- `Spread Mercado`: quanto o mercado está precificando acima do tesouro mais longo (geralmente fica em torno de 2% acima)
    
- Iremos usar o `WIDGETS` do Python para simular o `SPREAD_MERCADO`

In [92]:
# Criar 2 colunas com os dados já obtidos

fundos['NTN-B'] = spread_ntnb
fundos['IPCA'] = ipca

In [112]:
fundos.loc[fundos['Papel'].str.contains('OF','FF')]

,Papel,Segmento,Cotação,FFO Yield,Dividend Yield,P/VP,Valor de Mercado,Liquidez,Qtd de imóveis,Preço do m2,Aluguel por m2,Cap Rate,Vacância Média,Dividend yield 2,Dividendo/cota,NTN-B,IPCA,SPREAD,WACC,ValorJusto,Sinal
4,AFOF11,Títulos e Val. Mob.,224.00,"1,79%","0,00%",2.08,82235800,111274,0,0.00,0.00,"0,00%","0,00%",0.00,0.000000,4.21,6.76,2.0,0.1297,0.00,Acima valor justo
77,FMOF11,Lajes Corporativas,70.01,"1,45%","1,53%",0.53,35565600,3040,2,2106.94,174.95,"8,30%","39,48%",1.53,1.071153,4.21,0.00,2.0,0.0621,17.25,Acima valor justo
98,HFOF11,Títulos e Val. Mob.,99.00,"6,25%","5,78%",1.03,2133150000,4718620,0,0.00,0.00,"0,00%","0,00%",5.78,5.722200,4.21,6.76,2.0,0.1297,44.12,Acima valor justo
130,KFOF11,Títulos e Val. Mob.,91.80,"3,89%","5,01%",0.87,416158000,534500,0,0.00,0.00,"0,00%","0,00%",5.01,4.599180,4.21,6.76,2.0,0.1297,35.46,Acima valor justo
143,LOFT11B,Residencial,29.90,"-30,48%","46,49%",0.34,68980300,476,154,3784.48,26.51,"0,70%","0,00%",46.49,13.900510,4.21,0.00,2.0,0.0621,223.84,Abaixo valor justo
215,RFOF11,Títulos e Val. Mob.,82.80,"4,90%","5,02%",0.88,89272000,291128,0,0.00,0.00,"0,00%","0,00%",5.02,4.156560,4.21,6.76,2.0,0.1297,32.05,Acima valor justo
235,TBOF11,Lajes Corporativas,101.50,"-0,08%","54,42%",656.11,1020080000,0,0,0.00,0.00,"0,00%","0,00%",54.42,55.236300,4.21,0.00,2.0,0.0621,889.47,Abaixo valor justo


In [113]:
def modelo_gordon(fiis,spread_mercado):
    # filtro para zerar o IPCA de todos os FIIs que não sao de CRI
    fiis.loc[~(fiis['Segmento']=='Títulos e Val. Mob.')|
            (fiis['Papel'].str.contains('OF','FF')) ,'IPCA'] = 0
    fiis['SPREAD'] = spread_mercado
    fiis['WACC'] = (fiis['NTN-B'] + fiis['IPCA'] + fiis['SPREAD'])/100
    fiis['ValorJusto'] = round(fiis['Dividendo/cota'] / fiis['WACC'],2)
    fiis['Sinal'] = 'Acima valor justo'
    fiis.loc[(fiis['ValorJusto']-fiis['Cotação']) >= 0 ,'Sinal'] = 'Abaixo valor justo'
    a = fiis[['Papel','Cotação','ValorJusto','Sinal']]
    a = a.loc[a['Sinal'] == 'Abaixo valor justo']
    print(a)

In [114]:
# Slider do spread mercado
valor_spread = widgets.FloatSlider(value=2,min=0,max=2.5,step=0.1,description='SpreadMercado:',
    disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='.1f')
        
# WIDGET CREATED      
widgets.interactive(modelo_gordon,fiis = fixed(fundos),spread_mercado = valor_spread)

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='SpreadMercado:', max=2.5, r…